# EMBO Practical Course <br/>"Advanced methods in bioimage analysis"

***

Homepage: https://www.embl.org/about/info/course-and-conference-office/events/bia23-01/

***

## Day 2 - Session 1: Image Data Management - 10:15 to 11:15 "Get set"


<table style="table { position: relative;  display: inline-block; } img {  position: absolute;  left: 0;  right: 0;  width: auto;  height: 100%;  object-fit: cover;  object-position: center;}">
    <tr>
        <td style="vertical-align: top">
            <h3>Introduction</h3>
            <p>
                In this notebook, we'll look at issues arising from having many files
                (<b>and many file formats!</b>b>) on your file servers. A quick intro into working
                with POSIX file systems will be followed by tools for converting data into
                a common file-format: OME-TIFF.
            </p>
            <p>
                The two main goals are: (1) make sure you have the basics you might need
                for the rest of the course and (2) encourage you to explore these tools
                further on your own to speed up your day-to-day activities.
            </p>
            <p>
                Outline:
                <ol start="3">
                    <li>Bash & Scripting
                        <ol type="a">
                            <li>Bash basics</li>
                            <li>Our data directory</li>
                            <li>Extras: time permitting
                                <ul>
                                    <li>Permissions</li>
                                    <li>Simple scripting</li>
                                    <li>shasum</li>
                                    <li>h5tools</li>
                                </ul>
                            </li>
                        </ol>
                    </li>
                        <li>bftools
                        <ol type="a">
                            <li>showinf, bfconvert, etc.</li>
                            <li>Working with OME-TIFF</li>
                            <li>aicsimageio (time-permitting)</li>
                        </ol>
                    </li>
                </ol>
            </p>
        </td>
        <td>
            <center>
                <img src="images/falk/clara-shares-300dpi.png"/>
            </center>
            <center>
                <small>
                    <a href="https://github.com/zarr-developers/zarr-illustrations-falk-2022#clara-shares">"Clara shares"</a>
                    by Henning Falk, ©2022 NumFOCUS, is used under a CC BY 4.0 license. Modifications to this photo include cropping.
                </small>
            </center>
        </td>
    </tr>
</table>

### How to get started after the workshop?

[Binder](https://mybinder.org/) is a service for running notebooks online which can often be useful. In order to be be compatible with Binder, there is a folder `binder/` which contains the necessary files for configurig [this repo](https://github.com/ome/EMBO-Practical-Course-2023). The `environment.yml` defines a [Conda](https://docs.conda.io/en/latest/) environment which you can install yourself.

In [1]:
ls binder/

apt.txt  environment.yml


In [2]:
cat binder/environment.yml

channels:
  - ome
  - conda-forge
  - joshmoore
  - defaults
dependencies:
  - python ~= 3.9.0
  - notebook
  - nb_conda_kernels
  - napari
  - napari-aicsimageio
  - bioformats_jar
  - ipython
  - wheel
  - requests
  - h5utils
  - s3fs
  - scikit-image
  - scipy
  - xarray
  - zarr >= 2.4.0
  - bftools
  - bioformats2raw
  - raw2ometiff
  - omero-py
  - go-mc
  - pv
  - pip
  - pip:
      - PyQT5~=5.13.1
      - ome-zarr
      - awscli
      - omero-cli-zarr
      - napari-ome-zarr
      - xmlschema
      - bash_kernel


You can perform those actions on your own system _without_ a Jupyter notebook. For that, you will need to install the correct tools like `bioformats2raw`. Instructions are available under https://github.com/ome/EMBO-Practical-Course-2023/blob/main/README.md
 
*** 

## 3a. Bash basics

If you are managing files on a server, it will often be easier to use the command-line once you learn it. And if you are using a command-line, it's likely going to be Bash. There are a large number of useful commands, and it will take time to become familiar with them all, <b>but it is worth your time.</b>

Often, the shorter the name the more useful it is. Let's take `ls` for "list". To learn more about a tool, you can use the `man` tool, or any many cases by passing the `-h` or `--help` argument.

In [3]:
man ls

man: can't set the locale; make sure $LC_* and $LANG are correct
LS(1)                            User Commands                           LS(1)



NAME
       ls - list directory contents

SYNOPSIS
       ls [OPTION]... [FILE]...

DESCRIPTION
       List  information  about  the FILEs (the current directory by default).
       Sort entries alphabetically if none of -cftuvSUX nor --sort  is  speci-
       fied.

       Mandatory  arguments  to  long  options are mandatory for short options
       too.

       -a, --all
              do not ignore entries starting with .

       -A, --almost-all
              do not list implied . and ..

       --author
              with -l, print the author of each file

       -b, --escape
              print C-style escapes for nongraphic characters

       --block-size=SIZE
              scale sizes by SIZE before printing them; e.g., '--block-size=M'
              prints sizes in units of 1,048,576 bytes; see SIZE format below

       -B, --ignore

****

I bring up `ls` first because it can be used to find other commands:

In [4]:
ls -ltrad /bin /usr/bin/ /usr/local/bin

lrwxrwxrwx. 1 root root     7 Oct 30  2020 /bin -> usr/bin
drwxr-xr-x. 2 root root   131 Aug 22 13:16 /usr/local/bin
dr-xr-xr-x. 2 root root 61440 Aug 25 06:33 /usr/bin/


The "$PATH" (an environment variable) defines *all* of the places that commands can be found. (Don't worry. You don't need to do anything with all of these)

In [5]:
echo $PATH

/opt/munge-0.5.12/bin:/opt/slurm-21.08.8/bin:/opt/slurm-21.08.8/sbin:/opt/nhc-1.4.2/sbin:/scratch/bioimagecourse2023/session1/mambaforge/envs/embo/bin:/scratch/bioimagecourse2023/session1/mambaforge/condabin:/opt/munge-0.5.12/bin:/opt/slurm-21.08.8/bin:/opt/slurm-21.08.8/sbin:/opt/nhc-1.4.2/sbin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/nextflow:/scratch/bioimagecourse2023/session1:/opt/nextflow


These are the directories where you will find many of them. For example, the command `bash` is one of them:

You can use the `tr` command to "translate" characters, here from ":" to newlines

In [6]:
echo $PATH | tr : '\n'

/opt/munge-0.5.12/bin
/opt/slurm-21.08.8/bin
/opt/slurm-21.08.8/sbin
/opt/nhc-1.4.2/sbin
/scratch/bioimagecourse2023/session1/mambaforge/envs/embo/bin
/scratch/bioimagecourse2023/session1/mambaforge/condabin
/opt/munge-0.5.12/bin
/opt/slurm-21.08.8/bin
/opt/slurm-21.08.8/sbin
/opt/nhc-1.4.2/sbin
/usr/local/bin
/usr/bin
/usr/local/sbin
/usr/sbin
/opt/nextflow
/scratch/bioimagecourse2023/session1
/opt/nextflow


There are many more environment variables. You can see all of them by running "set"

In [7]:
set | head -n 10

ABRT_DEBUG_LOG=/dev/null
BASH=/usr/bin/bash
BASHOPTS=checkwinsize:cmdhist:expand_aliases:extglob:extquote:force_fignore:histappend:interactive_comments:progcomp:promptvars:sourcepath
BASH_ALIASES=()
BASH_ARGC=()
BASH_ARGV=()
BASH_CMDS=()
BASH_COMPLETION_COMPAT_DIR=/etc/bash_completion.d
BASH_ENV=/usr/share/lmod/lmod/init/bash
BASH_LINENO=()


`which` is a command that tells you where a command lives. `/bin/bash` is the command that gets run when you login. It's your "shell".

In [8]:
which bash

/usr/bin/bash


There's also an environment variable that tells you what your shell is:

In [9]:
echo $SHELL

/bin/bash


That *shell* (or interpreter) holds the state of everything you are doing, including any variables you might set. (This is also why Jupyter can be confusing.)

Any time you start a new shell, you will need to make sure all of those variables are reset. For example, most of our notebooks will make use of the variable `YOURNAME`, e.g., to create directories, etc. You can find out your login name with the `whoami` command:

In [10]:
whoami

josh_openmicroscopy


In [11]:
YOURNAME=$(whoami)

In [12]:
echo $YOURNAME

josh_openmicroscopy


Now we have the variable we want set.

## 3c. Our data directory

Now with this out of the way, we can start to work with the data directories that are available on BAND.

`pwd` prints the directory that the current process is in:

In [13]:
pwd

/scratch/bioimagecourse2023/session1/EMBO-Practical-Course-2023


We want to move to another directory, for that we use the `cd` command:

In [14]:
cd /scratch/bioimagecourse2023/session1

Now we are in our session directory. Feel free to look around using `ls`. For example the flags `-ltra` mean: "show me a long listing of the files in reverse order by time and even show me the weird files starting with `.`".

In [15]:
ls -ltra

total 22917
-r--r-----  1 josh_openmicroscopy users 23433941 Aug 29 10:44 OMEZarrReader-0.3.1-jar-with-dependencies.jar
-r-xr-xr-x  1 josh_openmicroscopy users       78 Aug 29 11:13 notebook.sh
dr-xr-xr-x 19 josh_openmicroscopy users     4096 Aug 29 14:53 mambaforge
-r--r--r--  1 josh_openmicroscopy users      751 Aug 29 14:56 .mambarc
-r--r--r--  1 josh_openmicroscopy users       38 Aug 31 09:17 .gitignore
-r--r--r--  1 josh_openmicroscopy users      676 Aug 31 10:44 README.md
dr-xr-xr-x  6 josh_openmicroscopy users     4096 Aug 31 10:48 data
dr-xr-xr-x  2 josh_openmicroscopy users     4096 Aug 31 14:12 bin
-r--r--r--  1 josh_openmicroscopy users      197 Aug 31 14:25 setup.sh
dr-xr-xr-x  7 josh_openmicroscopy users     4096 Aug 31 14:25 .
dr-xr-xr-x  8 josh_openmicroscopy users     4096 Sep  4 15:10 .git
drwxr-xr-x  3 josh_openmicroscopy users     4096 Sep  5 09:22 ..
drwxr-xr-x  8 josh_openmicroscopy users     4096 Sep 10 15:12 EMBO-Practical-Course-2023


There are many other flags you can use. "-S" flag means sort by size instead of time:

In [16]:
ls -lSr data/

total 17
-r--r--r-- 1 josh_openmicroscopy users    0 Aug 31 09:59 a.fake
-r--r--r-- 1 josh_openmicroscopy users  617 Aug 31 10:47 README.md
dr-xr-xr-x 3 josh_openmicroscopy users 4096 Aug 31 10:46 TIFF
dr-xr-sr-x 2 josh_openmicroscopy users 4096 Mar 15  2018 SVS
dr-xr-xr-x 3 josh_openmicroscopy users 4096 Aug 22 14:56 HCS
dr-xr-xr-x 2 josh_openmicroscopy users 4096 Mar 28  2018 DV


Another important tool for understanding the size of your data is `du`, for "disk usage". The `-sh` flags "say show me only a (s)ummary of the data and put it in (h)uman-readable form"

In [17]:
\du -sh data/

13G	data/


(The `\` in front of the `du` command means "ignore any aliases that are being used")

Now we want to make a directory for you to do all of your work in:
 * `mkdir -p`  means "create all parents" (but also: "don't fail if it also exists" which is useful for Jupyter notebooks!)
 * `test -e data ||` means "if the file or directory `data` does not exist, then create it by "linking" it into your own directory. This doesn't make a copy but does make it easier for you to find things.

### ln (symlinks)

In [18]:
mkdir -p /scratch/${YOURNAME}/session1
cd /scratch/${YOURNAME}/session1
test -e data || ln -s /scratch/bioimagecourse2023/session1/data data

In [19]:
ls -ltra /scratch/${YOURNAME}/session1/

total 274
lrwxrwxrwx 1 josh_openmicroscopy users     41 Aug 31 14:18 data -> /scratch/bioimagecourse2023/session1/data
drwxr-xr-x 4 josh_openmicroscopy users   4096 Aug 31 14:50 a.ome.zarr
drwxr-xr-x 3 josh_openmicroscopy users   4096 Sep 10 15:08 ..
drwx------ 3 josh_openmicroscopy users   4096 Sep 10 15:13 .
-rw-r--r-- 1 josh_openmicroscopy users 267691 Sep 10 15:13 a.ome.tiff


`find` is a very powerful tool that we can use to explore directories full of lots of files. (In this case, we've added `tail` to only show the last time lines of all the output).

### find

In [20]:
find data/ | tail -n 10

data/HCS/Operetta/59549/59549__2013-04-03T09_44_09-Measurement1/Images
data/HCS/Operetta/59549/59549__2013-04-03T09_44_09-Measurement1/Images/Index.ref.xml
data/HCS/Operetta/59549/59549__2013-04-03T09_44_09-Measurement1/Assaylayout
data/HCS/Operetta/59549/59549__2013-04-03T09_44_09-Measurement1/Assaylayout/Unnamed.xml
data/SVS
data/SVS/77928.svs
data/SVS/77917.svs
data/SVS/COPYING
data/SVS/readme.txt
data/a.fake


The `-name` flag can be used to match patterns on the filename. You can use `-iname` if you want to ignore the case of the files. The `wc` command counts words, lines, etc. The "-l" says print how many lines, i.e. how many TIFF files were found.

In [21]:
find data/ -name "*.tiff" | wc -l

2571


You can also run a command on every file you found. The `file` command says what type of a file the operating system *assumes* a file to be. (It's not always right.)

In [22]:
find data/SVS/ -type f -exec file {} \;

data/SVS/77928.svs: TIFF image data, little-endian
data/SVS/77917.svs: TIFF image data, little-endian
data/SVS/COPYING: UTF-8 Unicode text
data/SVS/readme.txt: ASCII text


## Excercise 1:
### Use the tools above to find some data you want to use and symlink data into your own folder for later use.

In [23]:
## Do something here or on the command-line. If you are having issues, ask.

## 3d. Extras (time permitting)

### Permissions

In the directory listings above, e.g.:

```
drwxr-xr-x   936 root     wheel  29952 Jul 11 10:56 /usr/bin/
drwxr-xr-x@   39 root     wheel   1248 Jul 11 10:56 /bin
drwxr-xr-x  2086 jamoore  admin  66752 Aug 20 18:35 /usr/local/bin
```

the info here is **critical**:

```
PERMISSIONS ---- USER     GROUP   SIZE MODIFIED     NAME
```

Figuring out your user and group is pretty easy. `whoami` from above is your user. `id` can tell you your groups:

In [24]:
id

uid=1558(josh_openmicroscopy) gid=1102(users) groups=1102(users)


Together we can start to piece together whether or not you can read or edit a file. From https://www.grymoire.com/Unix/Permissions.html :

```
Three sections after the "d" for directory marker:
+------------+------+-------+
| Permission | Octal| Field |
+------------+------+-------+
| rwx------  | 700  | User  |
| ---rwx---  | 070  | Group |
| ------rwx  | 007  | Other |
+------------+------+-------+

Each section can be in one of 8 states:
+-----+---+--------------------------+
| rwx | 7 | Read, write and execute  |
| rw- | 6 | Read, write              |
| r-x | 5 | Read, and execute        |
| r-- | 4 | Read,                    |
| -wx | 3 | Write and execute        |
| -w- | 2 | Write                    |
| --x | 1 | Execute                  |
| --- | 0 | no permissions           |
+------------------------------------+

Examples:
+------------------------+-----------+
| chmod u=rwx,g=rwx,o=rx | chmod 775 | For world executables files
| chmod u=rwx,g=rx,o=    | chmod 750 | For executables by group only
| chmod u=rw,g=r,o=r     | chmod 644 | For world readable files
| chmod u=rw,g=r,o=      | chmod 640 | For group readable files
| chmod u=rw,go=         | chmod 600 | For private readable files
| chmod u=rwx,go=        | chmod 700 | For private executables
+------------------------+-----------+
```

This is *especially* important when using a shared folder like `/scratch`!

In [25]:
ls -ltra /scratch/${YOURNAME}

total 12
drwxrwxrwx 69 root                root  4096 Sep 10 15:08 ..
drwxr-xr-x  3 josh_openmicroscopy users 4096 Sep 10 15:08 .
drwx------  3 josh_openmicroscopy users 4096 Sep 10 15:13 session1


In [26]:
chmod go-rx /scratch/${YOURNAME}/session1/

In [27]:
ls -ltra /scratch/${YOURNAME}

total 12
drwxrwxrwx 69 root                root  4096 Sep 10 15:08 ..
drwxr-xr-x  3 josh_openmicroscopy users 4096 Sep 10 15:08 .
drwx------  3 josh_openmicroscopy users 4096 Sep 10 15:13 session1


### shasum

In [28]:
for x in $(find data/ -type f | tail -n 10);
do
    sha1sum ${x}
done

8ef95b369e7f23a8d46327098c70430d66a03b6f  data/HCS/Operetta/59549/b8a3e30d-258c-46f1-b8fe-a96fb51b9af6/r12c01-1974092126.tiff.gz
c7e9dc169f48836492a1a0e338c074751bc1bcaf  data/HCS/Operetta/59549/b8a3e30d-258c-46f1-b8fe-a96fb51b9af6/r15c01-0062694384.tiff.gz
364497ce13411f8f9c6dcf1d145eeb0dee3b24c9  data/HCS/Operetta/59549/b8a3e30d-258c-46f1-b8fe-a96fb51b9af6/r16c01-1169526669.tiff.gz
3dcbe901190f6ece57b5ea246937fdd6feb53f8a  data/HCS/Operetta/59549/59549__2013-04-03T09_44_09-Measurement1/Images/Index.ref.xml
4025ebf6563b60d82182dc75e6b957e0251e6242  data/HCS/Operetta/59549/59549__2013-04-03T09_44_09-Measurement1/Assaylayout/Unnamed.xml
337fbe356624c0288a48f87b537e0d0b74d1b315  data/SVS/77928.svs
03e0322f8f46ba058b02ce0b5ab4c203df9a6a73  data/SVS/77917.svs
1167f0e28fe2db01e38e883aaf1e749fb09f9ceb  data/SVS/COPYING
faf783f348878c99f05ccf40e5ba78901a0e3149  data/SVS/readme.txt
da39a3ee5e6b4b0d3255bfef95601890afd80709  data/a.fake


### h5tools

In [29]:
h5ls -h

usage: h5ls [OPTIONS] file[/OBJECT] [file[/[OBJECT]...]
  OPTIONS
   -h, -?, --help  Print a usage message and exit
   -a, --address   Print raw data address.  If dataset is contiguous, address
                   is offset in file of beginning of raw data. If chunked,
                   returned list of addresses indicates offset of each chunk.
                   Must be used with -v, --verbose option.
                   Provides no information for non-dataset objects.
   -d, --data      Print the values of datasets
   --enable-error-stack
                   Prints messages from the HDF5 error stack as they occur.
   --follow-symlinks
                   Follow symbolic links (soft links and external links)
                   to display target object information.
                   Without this option, h5ls identifies a symbolic link
                   as a soft link or external link and prints the value
                   assigned to the symbolic link; it does not provide any
         

### Learning more

This is obviously just a starting point. Perhaps check out a [Carpentry lesson](https://software-carpentry.org/lessons/) like [The Unix Shell](https://swcarpentry.github.io/shell-novice/) for more information. I can highly recommend them.

<hr/>

## Half-time

<hr/>

In [30]:
##
## Setup & Sanity checks
##

YOURNAME=$(whoami)
WORKDIR=/scratch/${YOURNAME}/session1
test -e ${WORKDIR} || {
    echo Please run the steps above first.
    exit 1
}

In [31]:
cd /scratch/${YOURNAME}/session1

## 4a. bftools: showinf, bfconvert, etc.

bftools is a package of tools that can be downloaded from https://downloads.openmicroscopy.org/bio-formats/latest/artifacts/ or [conda](https://anaconda.org/ome/bftools) for working with bioimaging files from the command-line (`conda install -c ome bftools`).

One of the tools, `formatlist` simply lists all the supported file formats with their associated file endings:

In [32]:
formatlist


File pattern: can read (pattern)
Zip: can read (zip)
Animated PNG: can read, can write, can write multiple (png)
JPEG: can read, can write (jpg, jpeg, jpe)
SlideBook 7 SLD (native): can read (sldy)
Portable Any Map: can read (pbm, pgm, ppm)
Flexible Image Transport System: can read (fits, fts)
PCX: can read (pcx)
Graphics Interchange Format: can read (gif)
Windows Bitmap: can read (bmp)
IPLab: can read (ipl)
IVision: can read (ipm)
RCPNL: can read (rcpnl)
Deltavision: can read (dv, r3d, r3d_d3d, dv.log, r3d.log)
Medical Research Council: can read (mrc, st, ali, map, rec, mrcs)
Gatan Digital Micrograph: can read (dm3, dm4)
Gatan DM2: can read (dm2)
Bitplane Imaris: can read (ims)
Openlab RAW: can read (raw)
OME-XML: can read, can write, can write multiple (ome, ome.xml)
Leica Image File Format: can read (lif)
Audio Video Interleave: can read, can write, can write multiple (avi)
PICT: can read (pict, pct)
SPCImage Data: can read (sdt)
SPC FIFO Data: can read (spc, set)
Encapsulated Post

We've got several files from aa few of these types that we can look at.

In [33]:
showinf -nopix data/a.fake

Checking file format [Simulated data]
Initializing reader
FakeReader initializing data/a.fake
Initialization took 0.157s

Reading core metadata
filename = data/a.fake
Used files = [/mnt/gluster2/scratch/josh_openmicroscopy/session1/data/a.fake]
Series count = 1
Series #0 :
	Image count = 1
	RGB = false (1) 
	Interleaved = false
	Indexed = false (true color)
	Width = 512
	Height = 512
	SizeZ = 1
	SizeT = 1
	SizeC = 1
	Tile size = 512 x 512
	Thumbnail size = 128 x 128
	Endianness = intel (little)
	Dimension order = XYZCT (certain)
	Pixel type = uint8
	Valid bits per pixel = 8
	Metadata complete = true
	Thumbnail series = false
	-----
	Plane #0 <=> Z 0, C 0, T 0


Reading global metadata

Reading metadata


In [34]:
showinf -nopix -omexml-only data/a.fake

<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
   <Image ID="Image:0" Name="a">
      <Pixels BigEndian="false" DimensionOrder="XYZCT" ID="Pixels:0" Interleaved="false" SignificantBits="8" SizeC="1" SizeT="1" SizeX="512" SizeY="512" SizeZ="1" Type="uint8">
         <Channel ID="Channel:0:0" SamplesPerPixel="1">
            <LightPath/>
         </Channel>
         <MetadataOnly/>
      </Pixels>
   </Image>
</OME>



In [35]:
test -e a.ome.tiff && rm -rf a.ome.tiff || echo Not yet created

In [36]:
bfconvert data/a.fake a.ome.tiff

data/a.fake
FakeReader initializing data/a.fake
[Simulated data] -> a.ome.tiff [OME-TIFF]
	Converted 1/1 planes (100%)
[done]
3.046s elapsed (13.0+1573.0ms per plane, 1269ms overhead)


In [37]:
tiffcomment a.ome.tiff

<?xml version="1.0" encoding="UTF-8"?><!-- Warning: this comment is an OME-XML metadata block, which contains crucial dimensional parameters and other important metadata. Please edit cautiously (if at all), and back up the original data before doing so. For more information, see the OME-TIFF web site: https://docs.openmicroscopy.org/latest/ome-model/ome-tiff/. --><OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" Creator="OME Bio-Formats 7.0.0" UUID="urn:uuid:1c096a89-e1d7-4b61-9666-e9625a4c1b07" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd"><Image ID="Image:0" Name="a"><Pixels BigEndian="false" DimensionOrder="XYZCT" ID="Pixels:0" Interleaved="false" SignificantBits="8" SizeC="1" SizeT="1" SizeX="512" SizeY="512" SizeZ="1" Type="uint8"><Channel ID="Channel:0:0" SamplesPerPixel="1"><LightPath/></Channel><TiffData FirstC="0" FirstT="0" Firs

### Pattern files (many files)

Pattern files are a Bio-Formats functionality that lets you write short patterns like below which group together other files along different dimensions. You can read more under https://bio-formats.readthedocs.io/en/stable/formats/pattern-file.html

In [38]:
cat data/TIFF/cellprofiler/fruit-fly-cells/POS218.pattern/POS218.pattern

01_POS218_Z<00>_T<00>_<D,F,R>.TIF


In [39]:
showinf -nopix data/TIFF/cellprofiler/fruit-fly-cells/POS218.pattern/POS218.pattern

Checking file format [File pattern]
Initializing reader
FilePatternReader initializing data/TIFF/cellprofiler/fruit-fly-cells/POS218.pattern/POS218.pattern
TiffDelegateReader initializing /mnt/gluster2/scratch/josh_openmicroscopy/session1/data/TIFF/cellprofiler/fruit-fly-cells/POS218.pattern/01_POS218_Z00_T00_D.TIF
Reading IFDs
Populating metadata
Checking comment style
Populating OME metadata
TiffDelegateReader initializing /mnt/gluster2/scratch/josh_openmicroscopy/session1/data/TIFF/cellprofiler/fruit-fly-cells/POS218.pattern/01_POS218_Z00_T00_D.TIF
Reading IFDs
Populating metadata
Checking comment style
Populating OME metadata
TiffDelegateReader initializing /mnt/gluster2/scratch/josh_openmicroscopy/session1/data/TIFF/cellprofiler/fruit-fly-cells/POS218.pattern/01_POS218_Z00_T00_D.TIF
TiffDelegateReader initializing /mnt/gluster2/scratch/josh_openmicroscopy/session1/data/TIFF/cellprofiler/fruit-fly-cells/POS218.pattern/01_POS218_Z00_T00_D.TIF
TiffDelegateReader initializing /mnt/glu

### Series (many images)

In [40]:
showinf -nopix 'a&series=2.fake'

Initializing reader
FakeReader initializing a&series=2.fake
Initialization took 0.166s

Reading core metadata
filename = a&series=2.fake
Used files = [/mnt/gluster2/scratch/josh_openmicroscopy/session1/a&series=2.fake]
Series count = 2
Series #0 :
	Image count = 1
	RGB = false (1) 
	Interleaved = false
	Indexed = false (true color)
	Width = 512
	Height = 512
	SizeZ = 1
	SizeT = 1
	SizeC = 1
	Tile size = 512 x 512
	Thumbnail size = 128 x 128
	Endianness = intel (little)
	Dimension order = XYZCT (certain)
	Pixel type = uint8
	Valid bits per pixel = 8
	Metadata complete = true
	Thumbnail series = false
	-----
	Plane #0 <=> Z 0, C 0, T 0

Series #1 :
	Image count = 1
	RGB = false (1) 
	Interleaved = false
	Indexed = false (true color)
	Width = 512
	Height = 512
	SizeZ = 1
	SizeT = 1
	SizeC = 1
	Tile size = 512 x 512
	Thumbnail size = 128 x 128
	Endianness = intel (little)
	Dimension order = XYZCT (certain)
	Pixel type = uint8
	Valid bits per pixel = 8
	Metadata complete = true
	Thumbnail s

### In bioimaging, anything is possible

You can see a graphical explanation of the "many-to-many" problem in https://downloads.openmicroscopy.org/presentations/2013/fs-workshop-paris/#/7/1

<table>
    <tr>
        <td>
            <img src="images/many-to-many/one-to-one.png"/>
        </td>
    </tr>
    <tr>
        <td>
            <img src="images/many-to-many/one-to-many.png"/>
        </td>
    </tr>
    <tr>
        <td>
            <img src="images/many-to-many/many-to-many.png"/>
        </td>
    </tr>
</table>


## Excercise 2:
### Use the tools above to find inspect the data you chose, and convert it to OME-TIFF.

In [41]:
## Do something here or on the command-line. If you are having issues, ask.

## License
Copyright (C) 2023 German BioImaging. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.